### Pull Full Text Papers ###

In [1]:
import pandas as pd
from metapub import PubMedFetcher
import requests
from semanticscholar import SemanticScholar

2023-06-17 13:08:24 vanis-mbp.lan metapub.config[12681] WARNING NCBI_API_KEY was not set.


In [ ]:
s2_api_key = 'jvN8XitMY59Wtoqm3FZnc94qRHnsKw3z3hLNh7XB'
sch = SemanticScholar(api_key=s2_api_key)

# follow s3 link
full_text = requests.get("http://api.semanticscholar.org/datasets/v1/release/latest/dataset/s2orc",
                      headers={'x-api-key':'jvN8XitMY59Wtoqm3FZnc94qRHnsKw3z3hLNh7XB'}).json()


In [ ]:
full_text

### Parse Files ###

In [2]:
import duckdb
import pandas as pd

conn = duckdb.connect()

**Import List of Corpus ids**

In [3]:
df = pd.read_csv('final_corpuslist.csv')

t = tuple(df['corpusid'])
query = "select * from article_attributes where corpusid IN {}".format(t)

**Parse gzip files using duck db**

In [4]:
import time

seconds = time.time()

for i in range(1,31):
    print ("file", i)
    x = conn.execute("select * from '{}.json.gz' where corpusid in {}".format(i,t)).fetchdf()
    x.to_csv('{}.csv'.format(i))
    print("file",i,"time =", time.time() - seconds)
    
print("Seconds since epoch =", time.time() - seconds)

file 1
file 1 time = 162.98096108436584
file 2
file 2 time = 308.1070120334625
file 3
file 3 time = 455.190593957901
file 4
file 4 time = 601.0564742088318
file 5
file 5 time = 744.1327021121979
file 6
file 6 time = 888.029806137085
file 7
file 7 time = 1029.0420892238617
file 8
file 8 time = 1168.2937471866608
file 9
file 9 time = 1307.9308910369873
file 10
file 10 time = 1445.06458902359
file 11
file 11 time = 1585.545599937439
file 12
file 12 time = 1723.3107051849365
file 13
file 13 time = 1860.680430173874
file 14
file 14 time = 1999.46000623703
file 15
file 15 time = 2137.236243247986
file 16
file 16 time = 2277.1493961811066
file 17
file 17 time = 2419.8535721302032
file 18
file 18 time = 2557.130742073059
file 19
file 19 time = 2696.7790892124176
file 20
file 20 time = 2832.94287109375
file 21
file 21 time = 2967.3952622413635
file 22
file 22 time = 3103.0412271022797
file 23
file 23 time = 3239.1913640499115
file 24
file 24 time = 3376.8884251117706
file 25
file 25 time = 3510

**Parse all CVS to get json into dataframe columns**

In [3]:
import ast
from pandas.io.json import json_normalize
import time


def only_dict(d):
    return ast.literal_eval(d)


seconds = time.time()

for i in range(1,31):
    df = pd.read_csv(str(i) + '.csv')
    df = df[['corpusid','content']]
    A = json_normalize(df['content'].apply(only_dict).tolist())
    df = df[['corpusid']].join([A])
    df.to_csv('{}_full_text.csv'.format(i))

print("Seconds since epoch =", time.time() - seconds)



/var/folders/f3/1bf0zv1s7991qswb6sl4v9k00000gn/T/ipykernel_15713/3015115667.py:15: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  A = json_normalize(df['content'].apply(only_dict).tolist())


Seconds since epoch = 449.7636981010437


**Export into CSV and check final results**

In [4]:
csv = []

for i in range(1,31):
    csv.append(str(i) + '_full_text.csv')
    
df = pd.concat(map(pd.read_csv, csv))


df.head(5)

,Unnamed: 0,corpusid,text,source.pdfurls,source.pdfsha,source.oainfo,annotations.abstract,annotations.author,annotations.authoraffiliation,annotations.authorfirstname,...,annotations.paragraph,annotations.publisher,annotations.sectionheader,annotations.table,annotations.tableref,annotations.title,annotations.venue,source.oainfo.license,source.oainfo.openaccessurl,source.oainfo.status
0,0,250150147,\nNext-Generation Metagenome Sequencing Shows ...,NaN,a3d95f615d674c56efc6cc0e134a773951328c79,NaN,"[{""end"":4475,""start"":2322}]","[{""end"":254,""start"":245},{""end"":275,""start"":25...","[{""end"":494,""start"":385},{""end"":615,""start"":50...","[{""end"":248,""start"":245},{""end"":262,""start"":25...",...,"[{""end"":5260,""start"":4491},{""end"":6886,""start""...",NaN,"[{""end"":4489,""start"":4477},{""end"":9898,""start""...","[{""end"":37053,""start"":33349},{""end"":38156,""sta...","[{""attributes"":{""ref_id"":""tab_0""},""end"":15910,...","[{""end"":195,""start"":1},{""end"":1738,""start"":1544}]","[{""end"":1756,""start"":1740}]",NaN,NaN,NaN
1,1,251969342,\nProtective roles of cytoplasmic p21 Cip1/Waf...,NaN,77d34fefc0ce5bbf0e130f621720f9490e07a1d4,NaN,"[{""end"":3064,""start"":1441}]","[{""end"":270,""start"":99},{""end"":371,""start"":271...","[{""end"":198,""start"":115},{""end"":269,""start"":20...","[{""end"":104,""start"":99},{""end"":278,""start"":271...",...,"[{""end"":5305,""start"":3066},{""end"":5893,""start""...",NaN,"[{""attributes"":{""n"":""2.1""},""end"":5333,""start"":...",NaN,NaN,"[{""end"":96,""start"":1},{""end"":1271,""start"":1176}]",NaN,NaN,NaN,NaN
2,2,7063556,\nThe Impact of Nutritional Status and Longitu...,NaN,fcd362c9e4615d887d1600faad9b6e319422a57d,NaN,"[{""end"":2815,""start"":1706}]","[{""end"":402,""start"":136},{""end"":672,""start"":40...","[{""end"":312,""start"":161},{""end"":401,""start"":31...","[{""end"":140,""start"":136},{""end"":409,""start"":40...",...,"[{""end"":4127,""start"":2831},{""end"":4959,""start""...",NaN,"[{""attributes"":{""n"":""1.""},""end"":2829,""start"":2...",NaN,"[{""end"":14302,""start"":14295}]","[{""end"":129,""start"":1},{""end"":1380,""start"":1252}]","[{""end"":1417,""start"":1382}]",CCBY,https://www.mdpi.com/1660-4601/8/1/105/pdf,GOLD
3,3,14778566,\nARTICLE Coarse-grained simulation reveals ke...,NaN,4c80183d0ae3364fc2db2a9a024222cd2888d662,NaN,"[{""end"":2734,""start"":1751}]","[{""end"":284,""start"":109},{""end"":458,""start"":28...","[{""end"":283,""start"":125},{""end"":457,""start"":29...","[{""end"":113,""start"":109},{""end"":117,""start"":11...",...,"[{""end"":3226,""start"":2736},{""end"":4255,""start""...",NaN,"[{""end"":8028,""start"":8021},{""end"":23211,""start...","[{""end"":42963,""start"":42373},{""end"":45178,""sta...","[{""attributes"":{""ref_id"":""tab_0""},""end"":8877,""...","[{""end"":85,""start"":1},{""end"":1558,""start"":1474}]",NaN,CCBY,https://doi.org/10.1038/ncomms11568,GOLD
4,4,475102,\nPhysiological relation between respiration a...,NaN,a6c6bd1e378da5c7a0561f755808a38f282d8cbd,NaN,"[{""end"":3427,""start"":436}]","[{""end"":169,""start"":153},{""end"":172,""start"":17...",NaN,"[{""end"":159,""start"":153},{""end"":161,""start"":16...",...,"[{""end"":4323,""start"":3441},{""end"":5816,""start""...",NaN,"[{""end"":3439,""start"":3429},{""end"":7292,""start""...","[{""end"":38677,""start"":38670},{""end"":39006,""sta...","[{""attributes"":{""ref_id"":""tab_0""},""end"":7981,""...","[{""end"":150,""start"":1},{""end"":407,""start"":258}]",NaN,CCBY,https://microbialcellfactories.biomedcentral.c...,GOLD


In [5]:
len(df)

158844

In [6]:
df1 = pd.read_csv('final_corpuslist.csv')

set(df['corpusid']).issubset(set(df1['corpusid']))

True